# Objectif

Exemple d'utilisation d'un notebook Jupyter, permettant d'utiliser des commentaires, du texte, des jeux de données associées, des scripts.

J'en profite pour utiliser mon ensemble de fonctions (je n'ose parler de "librairie Python") pour manipuler des données du SRU : le fichier **[SRUextraction.py](https://github.com/Lully/bnf-sru/blob/master/SRUextraction.py)**

In [2]:
import SRUextraction as sru               # https://github.com/Lully/bnf-sru/blob/master/SRUextraction.py
from stdf import create_file, line2report #  https://github.com/Lully/bnf-sru/blob/master/stdf.py

Requête envoyée au SRU de la BnF : http://catalogue.bnf.fr/api/ : 

Notices de la Bibliographie nationale de France (rubrique "Livres") créées après le 1er janvier 2022 :

In [3]:
query = 'bib.frenchNationalBibliography all "Books" and bib.creationdate >= "20220101"'
url_sru = "http://catalogue.bnf.fr/api/SRU"

Premier lot de résultats, en utilisant l'objet SRU_result qui contient un certain nombre d'attributs, dont :
 * **SRU_result.nb_results** : nombre total de résultats
 * **SRU_result.list_identifiers** : liste des identifiants de notices
 * **SRU_result.dict_records** : dictionnaire des notices, dont la clé est l'identifiant et la valeur la notice en MarcXML

On crée un fichie de résultats dans lequel on envoie toutes les notices (enfin, plusieurs zones pour chaque notice) ligne à ligne.

Comme le SRU va envoyer les résultats 500 par 500, on pagine les pages de résultats pour aller jusqu'au dernier

Pour chaque ARK trouvé, on va récupérer la valeur de certaines zones grâce à la fonction **record2fieldvalue** qui accepte 2 paramètres :
 * la notice (en MarcXML)
 * la zone à récupérer (qui peut être sous la forme : `200`, `200$a` ou encore `200$a$f`, par exemple) 

In [5]:
report = create_file("livresBibNatFra2022.txt", ["ARK", "200", "214", "214$c", "700", "300$a"])
params = {"recordSchema": "unimarcxchange", "maximumRecords": "500"}
results = sru.SRU_result(query, url_sru, 
                         parametres=params)
for ark in results.dict_records:
    zone_200 = sru.record2fieldvalue(results.dict_records[ark], "200") # zone de titre
    zone_214 = sru.record2fieldvalue(results.dict_records[ark], "214") # Adresse de l'éditeur
    zone_214c = sru.record2fieldvalue(results.dict_records[ark], "214$c") # Nom de l'éditeur
    zone_700 = sru.record2fieldvalue(results.dict_records[ark], "700") # Lien aux auteurs
    zone_300a = sru.record2fieldvalue(results.dict_records[ark], "300")# zone de note
    line2report([ark, zone_200, zone_214, zone_214c, zone_700, zone_300a], report, display=False)


['ARK', '200', '214', '214$c', '700', '300$a']


Pour l'instant on n'a récupéré que la première page de 500 résultats. Il fautà présent paginer les résultats en demandant les pages 501 à 1000, 1001 à 1500, etc.

En l'occurrence, il y a environ 20.000 résultats, donc **le script doit lancer successivement 40 requêtes**

In [ ]:
for i in range(501, results.nb_results, 500):
    params["startRecord"] = str(i)
    results = sru.SRU_result(query, url_sru, 
                         parametres=params)
    for ark in results.dict_records:
        zone_200 = sru.record2fieldvalue(results.dict_records[ark], "200") # zone de titre
        zone_214 = sru.record2fieldvalue(results.dict_records[ark], "214") # Adresse de l'éditeur
        zone_214c = sru.record2fieldvalue(results.dict_records[ark], "214$c") # Nom de l'éditeur
        zone_700 = sru.record2fieldvalue(results.dict_records[ark], "700") # Lien aux auteurs
        zone_300a = sru.record2fieldvalue(results.dict_records[ark], "300")# zone de note
        line2report([ark, zone_200, zone_214, zone_214c, zone_700, zone_300a], report, display=False)


Ce script a permis de produire le fichier intitulé **livresBibNatFra2022.txt** situé dans le même répertoire

[livresBibNatFra2022.txt](./livresBibNatFra2022.txt)